# Analiza zbioru: Absenteeism at work
Bartłomiej Błaszczyk 236382/209276

# Spis treści:

- [1. Eksploracja danych](#1-eksploracja-danych)
    - [1.1 Wczytanie danych](#11-wczytanie-danych)
    - [1.2 Wstępny podgląd danych](#12-wstępny-podgląd-danych)
    - [1.3 Podsumowanie statystyczne](#13-podsumowanie-statystyczne)
    - [1.4 Identyfikacja brakujących danych](#14-identyfikacja-brakujących-danych)
    - [1.5 Identyfikacja cech](#15-identyfikacja-cech)
    - [1.6 Dystrybucja danych](#16-dystrybucja-danych)
    - [1.7 Relacje między cechami](#17-relacje-między-cechami)
    - [1.8 Wartości odstające](#18-wartości-odstające)
    - [1.9 Obserwacje](#19-obserwacje)
- [2. Przygotowanie danych do treningu](#2-przygotowanie-danych-do-treningu)
    - [2.1 Normalizacja danych](#21-normalizacja-danych)
        - [2.1.1 Normalizacja Z-score](#211-normalizacja-z-score-z-użyciem-skalowania-cech-o-dystrybucji-zbliżonej-do-standardowej)
        - [2.1.2 Normalizacja Min-Max](#212-normalizacja-min-max-dla-cech-o-rozkładzie-nienormalnym-xd)
        - [2.1.3 Kodowanie kategorii](#213-kodowanie-cech-kategorycznych)
    - [2.2 Podział na zbiory treningowe i testowe](#22-podział-danych-na-zbiory-testowe-i-treningowe)
- [3. Modelowanie](#3-modelowanie)
    - [3.1 Pipeline](#31-tworzenie-lejka-danych-do-modelu)
    - [3.2 Ewaluacja](#32-ewaluacja-modelu)
        - [3.2.1 Confusion Matrix](#321-confusion-matrix)
        - [3.2.2 F1-Score](#322-f1-score)
        - [3.2.3](#323-roc-auc-score)
        - [3.2.4](#324-wnioski)
- [4. Konkluzje](#4-konkluzje)

# Zrozumienie biznesu
## Q: Jakie wyzwanie biznesowe chcesz rozwiązać?
### A: Zidentyfikować najważniejszą cechę, która prowadzi do porażki dyscyplinarnej.
## Q: Dlaczego to jest ważne?
### A: Pomoże to zespołowi reagować lepiej na niepokojące oznaki prowadzące do zwolnienia pracownika.
## Q: Co starasz się osiągnąć i jak to będzie mierzone?
### A: Chcę doprowadzić do zmniejszenia liczby zwolnień w zesppole.
## Q: Ile masz czasu na projekt?
### A: Tydzień.
## Q: Jak wykorzystasz model/analizy, aby osiągnąć zakładany cel?
### A: Przedstawię wskazówki zespołowi HR.

# 1. Eksploracja danych

## 1.1 Wczytanie danych

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

raw_data_path = "../data/raw/absenteeism_at_work.csv"
df = pd.read_csv(raw_data_path, delimiter=";")

## 1.2 Wstępny podgląd danych
- Rozmiary tabeli.
- Pierwsze i ostatnie rzędy.

In [ ]:
print(f"Dataset contains {df.shape[0]} rows and {df.shape[1]} columns.")
df.head()

In [ ]:
df.tail()

## 1.3 Podsumowanie statystyczne
- Dane statystyczne.
- Metadane o tabeli.

In [ ]:
df.describe()

In [ ]:
df.info()

## 1.4 Identyfikacja brakujących danych.

In [ ]:
missing_values = df.isnull().sum()
print(missing_values)

Brak brakujących danych

## 1.5 Identyfikacja cech

Podczas identyfikacji tworzę kopię zbioru df (Data Frame) do zmiennej dfc (Data Frame Categorical), żeby móc łatwiej rozpoznać zależności na wykresach.

In [ ]:
import textwrap
dfc = df.copy()

df.hist(bins=30, figsize=(15, 10))
plt.suptitle('Generyczne histogramy')
plt.show()

icd_mapping = {
    1: "Certain infectious and parasitic diseases",
    2: "Neoplasms",
    3: "Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism",
    4: "Endocrine, nutritional and metabolic diseases",
    5: "Mental and behavioural disorders",
    6: "Diseases of the nervous system",
    7: "Diseases of the eye and adnexa",
    8: "Diseases of the ear and mastoid process",
    9: "Diseases of the circulatory system",
    10: "Diseases of the respiratory system",
    11: "Diseases of the digestive system",
    12: "Diseases of the skin and subcutaneous tissue",
    13: "Diseases of the musculoskeletal system and connective tissue",
    14: "Diseases of the genitourinary system",
    15: "Pregnancy, childbirth and the puerperium",
    16: "Certain conditions originating in the perinatal period",
    17: "Congenital malformations, deformations and chromosomal abnormalities",
    18: "Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified",
    19: "Injury, poisoning and certain other consequences of external causes",
    20: "External causes of morbidity and mortality",
    21: "Factors influencing health status and contact with health services",
    22: "Patient follow-up",
    23: "Medical consultation",
    24: "Blood donation",
    25: "Laboratory examination",
    26: "Unjustified absence",
    27: "Physiotherapy",
    28: "Dental consultation"
}

icd_order = list(icd_mapping.values())

months_mapping = {
    1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'
}

months_order = list(months_mapping.values())

days_mapping = {
    2: 'Monday', 3: 'Tuesday', 4: 'Wednesday', 5: 'Thursday', 6: 'Friday',
    7: 'Saturday', 1: 'Sunday'
}

days_order = list(days_mapping.values())

seasons_mapping = {
    1: 'Summer', 2: 'Autumn', 3: 'Winter', 4: 'Spring'
}

seasons_order = list(seasons_mapping.values())

education_mapping = {
    1: 'High school', 2: 'Graduate', 3: 'Postgraduate', 4: 'Master and Doctor'
}

education_order = list(education_mapping.values())

binary_mapping = {
    0: 'No', 1: 'Yes'
}

binary_order = ['No', 'Yes']

dfc['Reason for absence'] = dfc['Reason for absence'].replace(icd_mapping)
dfc['Reason for absence'] = pd.Categorical(dfc['Reason for absence'], categories=icd_order, ordered=True)

dfc['Month of absence'] = dfc['Month of absence'].replace(months_mapping)
dfc['Month of absence'] = pd.Categorical(dfc['Month of absence'], categories=months_order, ordered=True)

dfc['Day of the week'] = dfc['Day of the week'].replace(days_mapping)
dfc['Day of the week'] = pd.Categorical(dfc['Day of the week'], categories=days_order, ordered=True)

dfc['Seasons'] = dfc['Seasons'].replace(seasons_mapping)
dfc['Seasons'] = pd.Categorical(dfc['Seasons'], categories=seasons_order, ordered=True)

dfc['Education'] = dfc['Education'].replace(education_mapping)
dfc['Education'] = pd.Categorical(dfc['Education'], categories=education_order, ordered=True)

dfc['Disciplinary failure'] = dfc['Disciplinary failure'].replace(binary_mapping)
dfc['Disciplinary failure'] = pd.Categorical(dfc['Disciplinary failure'], categories=binary_order, ordered=True)

dfc['Social drinker'] = dfc['Social drinker'].replace(binary_mapping)
dfc['Social drinker'] = pd.Categorical(dfc['Social drinker'], categories=binary_order, ordered=True)

dfc['Social smoker'] = dfc['Social smoker'].replace(binary_mapping)
dfc['Social smoker'] = pd.Categorical(dfc['Social smoker'], categories=binary_order, ordered=True)


plt.figure(figsize=(14, 8))
ax = dfc['Reason for absence'].value_counts().reindex(dfc['Reason for absence'].cat.categories).plot(kind='bar')
plt.title('Histogram for Reason for Absence')
plt.xlabel('Reason for Absence')
plt.ylabel('Count')

labels = [textwrap.fill(label, 30) for label in dfc['Reason for absence'].cat.categories]
ax.set_xticklabels(labels, rotation=90, ha='right', fontsize=8)
plt.tight_layout()
plt.show()

categorical_columns = ['Day of the week', 'Seasons', 'Month of absence', 'Education', 'Disciplinary failure', 'Social drinker', 'Social smoker']

n_cols = 4
n_rows = (len(categorical_columns) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4 * n_rows))

axes = axes.flatten()

for i, column in enumerate(categorical_columns):
    ax = axes[i]
    dfc[column].value_counts().reindex(dfc[column].cat.categories).plot(kind='bar', ax=ax)
    ax.set_title(f'Histogram for {column}')
    ax.set_xlabel(column)
    ax.set_ylabel('Count')

    labels = [textwrap.fill(label, 30) for label in dfc[column].cat.categories]
    ax.set_xticklabels(labels, rotation=15, ha='right', fontsize=7)

for j in range(len(categorical_columns), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

## 1.6 Dystrybucja danych

In [ ]:
num_columns = len(df.select_dtypes(include=['float64', 'int64']).columns)
fig, axes = plt.subplots(nrows=(num_columns // 3) + 1, ncols=3, figsize=(15, 10))  # Adjusting grid size
axes = axes.flatten()

for i, column in enumerate(df.select_dtypes(include=['float64', 'int64']).columns):
    sns.boxplot(x=df[column], ax=axes[i])
    axes[i].set_title(f'Box Plot of {column}')

plt.tight_layout()
plt.show()

## 1.7 Relacje między cechami

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

colors = [(1, 0.8, 0.8), (0, 0, 0), (1, 0.8, 0.8)]
n_bins = 100
cmap_name = 'pink_white_pink'
cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

correlation_matrix = df.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap=cmap, vmin=-1, vmax=1, center=0, fmt=".2f")
plt.title('Mapa cieplna korelacji')
plt.show()

Na wykresie korelacji nie widać zależności, ktróre mogły by wskazywać na silne powiązania między dwoma cechami

## 1.8 Wartości odstające

In [ ]:
z_scores = np.abs(stats.zscore(df.select_dtypes(include=[np.number])))
outliers = (z_scores > 3).sum(axis=0)
print(f'Liczba wartości odstających dla danej cechy:\n{outliers}')

## 1.9 Obserwacje

### 1.9.1 Reason for absence i Disciplinary failure

Źródło danych nie opisuje kategorii 'Reason_for_absence_0', ale można wnioskować, że chodzi o nieobecność spowodowaną zwolnieniem. Trenując model należy wykluczyć 'Reason_for_absence_0' ze zbioru.

In [ ]:
crosstab_data = pd.crosstab(df['Reason for absence'], df['Disciplinary failure'])

plt.figure(figsize=(14, 8))
crosstab_data.plot(kind='bar', stacked=True, color=['skyblue', 'orange'])

plt.title('Stacked Bar Plot of Reasons for Absence by Disciplinary Failure')
plt.xlabel('Reason for Absence')
plt.ylabel('Number of Occurrences')
plt.xticks(rotation=45, ha='right', fontsize=8)
plt.legend(title='Disciplinary Failure', loc='upper right')

plt.tight_layout()
plt.show()

# 2 Przygotowanie danych do treningu

## 2.1 Normalizacja danych

### 2.1.1 Standaryzacja (normalizacja Z-score):

Podczas korzystania ze StandardScaler (normalizacji Z-score), dane są przekształcane w taki sposób, że:
Średnia skalowanych danych wynosi 0.
Odchylenie standardowe wynosi 1.
Odpowiednia, gdy dane są rozkładem normalnym.

### 2.1.2 Skalowanie min-max:

Skaluje dane do ustalonego zakresu, zazwyczaj [0, 1].
Odpowiednia, gdy dane nie mają rozkładu normalnego.

### Obsługa zmiennych kategorycznych

Modele nie mogą bezpośrednio obsługiwać zmiennych kategorycznych. Kodowanie pozwala na przekształcenie tych zmiennych w liczby, które mogą być użyte przez algorytmy takie jak regresja liniowa, drzewa decyzyjne, czy sieci neuronowe. Za pomocą przekształcenia kategorii danej cechy w kolumny typu binarnego możemy zakodować informację o występowaniu danej kategorii. Tworzy to dodatkową liczbę cech reprezentujące wszystkie kategorie. Takie kodowanie nazywa się One-Hot.


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
df_scaled = df.copy()

z_score_features = ['Distance from Residence to Work', 'Service time', 'Age', 'Son', 'Pet', 'Weight', 'Height', 'Body mass index', 'Work load Average/day ']
min_max_features = ['Transportation expense', 'Hit target', 'Absenteeism time in hours']
categorical_features = ['Reason for absence', 'Day of the week', 'Seasons', 'Month of absence', 'Education', 'Disciplinary failure', 'Social drinker', 'Social smoker']

df_scaled.columns

### 2.1.1 Normalizacja Z-score z użyciem skalowania cech o dystrybucji zbliżonej do standardowej

In [ ]:
z_scaler = StandardScaler()
df_scaled[z_score_features] = z_scaler.fit_transform(df[z_score_features])
df_scaled.head()

### 2.1.2 Normalizacja Min-Max dla cech o rozkładzie nienormalnym xD

In [ ]:
mm_scaler = MinMaxScaler()
df_scaled[min_max_features] = mm_scaler.fit_transform(df[min_max_features])
df_scaled.head()

### 2.1.3 Kodowanie cech kategorycznych

Użycie drop_first=True:

Używanie drop_first=True jest wskazne pracując z modelami o charakterystyce liniowej, gdy kategorie rozbite na poszczególne cechy tworzą oczywiste zależności liniowe między sobą. Pozbycie się pierwszej kategori z każdej cechy pprzy rozbiciu pozwala zminimalizować wpływ zaistnienia tego zjawiska przy modelowaniu liniowym.

In [ ]:
df_scaled = pd.get_dummies(df_scaled, columns=categorical_features, drop_first=True)
df_scaled.head()

Wszystkie kategorie zostały rozbite na cechy. Zbiór danych teraz posiada dodatkowe kolumny które reprezentują obecność lub brak kategori.

In [ ]:
df_scaled.info()

## 2.2 Podział danych na zbiory testowe i treningowe

Disciplinary failure jest zmienną którą będę próbował przewidzieć.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

Ze wzgledu na użycie narzędzia ColumnTransformer przy modelowaniu, nie będę używał ręcznie stworzonej zmiennej ```df_scaled``` do podziału na zbiory.

Przed podziałem następuje usunięcie kolummny ID, oraz targetu ze zbioru treningowego, oraz wybranie cechy jako "target".

Usuwam ze zbioru również cechę 'Reason for absence', ze względu na to, że kategoria 'Reason_for_absence_0' prawie bezpośrednio implikuje wystąpienie 'Disciplinary_failure_true'

Po wybraniu cech następuje podział 80/20 do treningu i testowania modelu.

In [ ]:
X = df.drop(columns=['Disciplinary failure', 'ID', 'Reason for absence', 'Absenteeism time in hours'])
y = df['Disciplinary failure']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Zdefiniowanie cech liczbowych i kategorycznych

In [ ]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# 3 Modelowanie

Ze względu na różnorodność danych i brak możliwości skalowania do jednej wartosci wybrałem model random forest, który jest dobry do klasyfikacji binarnej - Disciplinary failure (tak/nie).


## 3.1 Tworzenie lejka danych do modelu

Uzycie zmiennej drop='first' nie szkodzi użyciu modelu Random Forest, a pozwoli na skuteczne użycie Regresji Liniowej.

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])


pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])


In [ ]:
pipeline.fit(X_train, y_train)

## 3.2 Ewaluacja modelu

In [ ]:
y_pred = pipeline.predict(X_test)
y_prob = pipeline.predict_proba(X_test)[:, 1]

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

roc_auc = roc_auc_score(y_test, y_prob)
print(f"ROC-AUC Score: {roc_auc:.2f}")

### 3.2.1 Confusion Matrix

(TN): 139 przypadków, w których model poprawnie przewidział "Brak naruszenia dyscypliny."
(FP): 1 przypadek, w którym model błędnie przewidział "Naruszenie dyscypliny", kiedy go nie było.
(FN): 0 przypadków, w których model błędnie przewidział "Brak naruszenia dyscypliny", kiedy ono wystąpiło.
(TP): 8 przypadków, w których model poprawnie przewidział "Naruszenie dyscypliny."

Wszystkie 139 negatywnych przewidywań jest poprawnych, a tylko 1 jest niepoprawnie sklasyfikowane.
Model poprawnie zidentyfikował 99% wszystkich faktycznych negatywów.
Spośród wszystkich przypadków przewidzianych jako "True" (Naruszenie dyscypliny), 89% jest poprawnych.
Model poprawnie zidentyfikował wszystkie faktyczne przypadki "True" (8 na 8).

### 3.2.2 F1-Score

Łączy precyzję i czułość w jednym wskaźniku. Dla "False" (Brak naruszenia dyscypliny) wynosi 1.00, a dla "True" (Naruszenie dyscypliny) wynosi 0.94. Średnia F1 macro wynosi 0.97, a średnia ważona F1 wynosi 0.99, co wskazuje na dobrą równowagę między precyzją a czułością.

### 3.2.3 ROC-AUC score

Wynik ROC-AUC 1.00 wskazuje na idealne rozdzielenie klas. Oznacza to, że model doskonale rozróżnia "Naruszenie dyscypliny" i "Brak naruszenia dyscypliny."
The overall accuracy is 99%, indicating the model correctly predicts 99% of the cases.
Imbalanced Data Consideration:

### 3.2.4 Wnioski

Tylko 8 z 148 próbek ma "Naruszenie dyscypliny" (True). Sugeruje to, że zbiór danych jest silnie niezrównoważony, z bardzo nielicznymi przypadkami "Naruszenia dyscypliny". Choć model wykazuje doskonałą czułość dla klasy mniejszościowej (wszystkie 8 przypadków zostało poprawnie zidentyfikowanych), może to wynikać z tego, że model skutecznie nauczył się wzorców ze względu na mały rozmiar klasy mniejszościowej.

Model działa wyjątkowo dobrze na bieżącym zbiorze danych, ale należy zachować ostrożność w kontekście przeuczenia, zwłaszcza ze względu na niezrównoważenie klas. Zaleca się dalszą ocenę przy użyciu różnych zbiorów danych lub walidacji krzyżowej, aby potwierdzić odporność modelu.

## 3.3 Klasyfikacja istotności cech modelu

In [ ]:
feature_importances = pipeline.named_steps['classifier'].feature_importances_
feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance from Random Forest')
plt.gca().invert_yaxis()
plt.show()

# 4. Konkluzje

Najważniejszym predyktorem porażnki w dyscyplinowaniu pracownika jest fakt, czy miał już on na koncie dużo godzin nieobecności. Jest to oczywisty wniosek, dlatego lepiej się przyjżeć wynikom modelu usuwając z parametrów nieobecnośc w godzinach w kroku 2.2:

```X = df.drop(columns=['Disciplinary failure', 'ID', 'Reason for absence', 'Absenteeism time in hours'])```

Po usunięciu cechy widzimy wciąż bardzo dobre wyniki modelu. Patrząc na wykres współczynników istotności możemy domyślać się, że:
- „Dzień tygodnia” pozostaje kluczowym predyktorem, prawdopodobnie dlatego, że spotkania lub rozmowy dotyczące działań dyscyplinarnych lub zwolnień są strategicznie planowane na konkretne dni, takie jak piątki lub poniedziałki. Może to być działanie mające na celu zarządzanie wpływem na morale zespołu lub codzienne operacje.
- Wysokie średnie obciążenie pracą dziennie może prowadzić do wypalenia zawodowego, niezadowolenia lub obniżonego morale pracowników, co zwiększa prawdopodobieństwo problemów dyscyplinarnych. Z kolei niskie obciążenie pracą może oznaczać słabą wydajność lub brak zaangażowania, co również może prowadzić do negatywnych konsekwencji dla pracowników.
- Umiejętność osiągania wyznaczonych celów jest kluczowa. Pracownicy, którzy regularnie nie realizują tych celów, mogą spotkać się z konsekwencjami, w tym działaniami dyscyplinarnymi lub ostatecznym zwolnieniem. To wyraźny wskaźnik, że wskaźniki wydajności są ściśle monitorowane i cenione przez organizację.
- Wyższe koszty transportu mogą zniechęcać pracowników do regularnego przychodzenia do pracy, zwłaszcza jeśli koszty te są znaczące w stosunku do ich dochodów. Może to prowadzić do nieobecności, które pośrednio mogą przyczynić się do niepowodzeń dyscyplinarnych lub decyzji dotyczących zwolnień.
- Znaczenie „Miesiąca” może być związane ze zmianami sezonowymi, przeglądami końcoworocznymi lub cyklami budżetowymi, które zbieżają się z redukcjami etatów lub zwiększoną kontrolą wydajności pracowników. Może również pokrywać się z czasem wdrażania zmian organizacyjnych, takich jak restrukturyzacja lub wprowadzanie nowych polityk.
